In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import scipy.stats as stat
import pylab 
import math
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as LGR,Lasso
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [8]:
Fe_CAAC_dataset = df = pd.read_excel("F:\\Nitrogen_activation\\rajaraman\\office\\ofifice\\jupyter\\Extension\\T07\\heatmap\\agostic\\xgboost\\N2_activation_extension_T07_heatmap_agostaic_descrptors.xlsx")

In [9]:
df_1=df.drop(['F00','T01','T02','T03',],axis=1)
df_2=df_1.drop([0,18,9])
df_3=df_1.drop([0,18,9,8,26,17])
df_4=df_1.drop([8,26,17])

In [11]:
X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=20)


scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)
# transform train and test sets
X_train_scaled_MM = scaler.transform(X_train)
X_test_scaled_MM = scaler.transform(X_test)
X_train_scaled_MM = pd.DataFrame(X_train_scaled_MM, columns=X_train.columns)
X_test_scaled_MM = pd.DataFrame(X_test_scaled_MM, columns=X_test.columns)

SV_reg_model = SVR()

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'epsilon': [0.1, 0.2, 0.5, 1],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']  # 'scale' is the default, 'auto' uses 1/n_features
}

# Set up Grid Search
grid_search = GridSearchCV(estimator=SV_reg_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
#grid_search.fit(X_train, y_train)
grid_search.fit(X_train_scaled_MM,y_train)
# Best parameters found

#print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)



Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best score:  -183.32523633444455
Best parameters found:  {'C': 100, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}


In [12]:
SV_reg_model = SVR(C=best_params['C'],epsilon=best_params['epsilon'],kernel=best_params['kernel'],gamma=best_params['gamma'])

SV_reg_model.fit(X_train_scaled_MM,y_train)

# prediction on Training data
training_data_prediction_X_train_SV_reg = SV_reg_model.predict(X_train_scaled_MM)
# prediction on Test data
test_data_prediction_X_test_SV_reg = SV_reg_model.predict(X_test_scaled_MM)

# R squared Error
error_score_r2 = metrics.r2_score(y_train, training_data_prediction_X_train_SV_reg)
error_score_MAE = metrics.mean_absolute_error(y_train, training_data_prediction_X_train_SV_reg)
error_score_MSE = metrics.mean_squared_error(y_train, training_data_prediction_X_train_SV_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_train,training_data_prediction_X_train_SV_reg)))
# R squared Error
error_score_r2 = metrics.r2_score(y_test, test_data_prediction_X_test_SV_reg)
error_score_MAE = metrics.mean_absolute_error(y_test, test_data_prediction_X_test_SV_reg)
error_score_MSE = metrics.mean_squared_error(y_test, test_data_prediction_X_test_SV_reg)
print("R squared Error : ", error_score_r2)
print("mean_absolute_error : ", error_score_MAE)
print("mean_squared_error : ", error_score_MSE)
print("RMSE",np.sqrt(mean_squared_error(y_test,test_data_prediction_X_test_SV_reg)))


R squared Error :  0.9000202115093441
mean_absolute_error :  8.786424693779374
mean_squared_error :  174.82651608287253
RMSE 13.222197853718288
R squared Error :  0.850010534415879
mean_absolute_error :  10.21065520639785
mean_squared_error :  106.1930525281612
RMSE 10.305001335670035


In [13]:
# can tune other metrics, such as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
# fit the scaler to the train set, it will learn the parameters
scaler.fit(X)
# transform train and test sets
X_scaled_MM = scaler.transform(X)
X_scaled_MM = pd.DataFrame(X_scaled_MM, columns=X.columns)

scores = cross_val_score(SV_reg_model, X_scaled_MM, y, cv=2)
print(scores)
print(np.mean(scores))

scores = cross_val_score(SV_reg_model, X_scaled_MM, y, cv=3)
print(scores)
print(np.mean(scores))

scores = cross_val_score(SV_reg_model, X_scaled_MM, y, cv=4)
print(scores)
print(np.mean(scores))

scores = cross_val_score(SV_reg_model, X_scaled_MM, y, cv=5)
print(scores)
print(np.mean(scores))



[0.85946409 0.84528971]
0.8523768980862332
[0.83858394 0.89203557 0.86024009]
0.86361986512618
[0.69964927 0.886466   0.96129132 0.81124178]
0.8396620911539776
[0.55974814 0.9312409  0.78140246 0.95886034 0.73027864]
0.7923060980652434


In [15]:

# Initialize the Leave-One-Out cross-validator
loo = LeaveOneOut()

X = df_3.drop(['T04','F01','F02','F04','F06','F04','F05','F09','F07','F10',],axis=1)
y = df_3['T04']# loading the linear regression model


# Apply MinMax Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Scale the features and store them in X_scaled


# Initialize lists to store predictions and actual values
y_pred_loo = []
y_true_loo = []

# Loop over each training/test split
for train_index, test_index in loo.split(X_scaled):
    X_train_loo, X_test_loo = X_scaled[train_index], X_scaled[test_index]
    y_train_loo, y_test_loo = y.iloc[train_index], y.iloc[test_index]
    
    # Train the Lasso model
SV_reg_model = SVR(C=best_params['C'],epsilon=best_params['epsilon'],kernel=best_params['kernel'],gamma=best_params['gamma'])
SV_reg_model.fit(X_train_loo, y_train_loo)
    
# Predict the left-out sample
y_pred_loo.append(SV_reg_model.predict(X_test_loo)[0])
y_true_loo.append(y_test_loo.values[0])

# Convert lists to arrays
y_pred_loo = np.array(y_pred_loo)
y_true_loo = np.array(y_true_loo)

# Calculate LOO performance metrics
Q2_loo = r2_score(y_true_loo, y_pred_loo)
MAE_loo = mean_absolute_error(y_true_loo, y_pred_loo)
MSE_loo = mean_squared_error(y_true_loo, y_pred_loo)
RMSE_loo = np.sqrt(MSE_loo)

# Print the results
print("LOO Q^2 score: ", Q2_loo)
print("LOO Mean Absolute Error: ", MAE_loo)
print("LOO Mean Squared Error: ", MSE_loo)
print("LOO Root Mean Squared Error: ", RMSE_loo)



LOO Q^2 score:  nan
LOO Mean Absolute Error:  2.6729102425262568
LOO Mean Squared Error:  7.144449164601773
LOO Root Mean Squared Error:  2.6729102425262568


c:\users\nettemchandra sekhar\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
